In [75]:
# Load library
import datetime
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text

# Load constants
DATA_PATH = "data/hand-sign-images"
TENSORBOARD_PATH = "tensorboard"

In [27]:
tf.__version__

'2.10.0'

# Checking Data

In [22]:
!tree /a

Folder PATH listing for volume Program and Files
Volume serial number is 081B-4D30
D:.
+---.ipynb_checkpoints
\---data
    +---.ipynb_checkpoints
    \---hand-sign-images
        +---Test
        |   +---A
        |   +---B
        |   +---C
        |   +---D
        |   +---E
        |   +---F
        |   +---G
        |   +---H
        |   +---I
        |   +---K
        |   +---L
        |   +---M
        |   +---N
        |   +---O
        |   +---P
        |   +---Q
        |   +---R
        |   +---S
        |   +---T
        |   +---U
        |   +---V
        |   +---W
        |   +---X
        |   \---Y
        \---Train
            +---A
            +---B
            +---C
            +---D
            +---E
            +---F
            +---G
            +---H
            +---I
            +---K
            +---L
            +---M
            +---N
            +---O
            +---P
            +---Q
            +---R
            +---S
            +---T
            +---U
  

In [51]:
dataset_train = tf.keras.utils.image_dataset_from_directory(
    directory=f"{DATA_PATH}/Train",
    color_mode='grayscale',
    label_mode='categorical',
    batch_size=32,
    image_size=(28, 28),
    seed=2023,
)
dataset_test = tf.keras.utils.image_dataset_from_directory(
    directory=f"{DATA_PATH}/Test",
    color_mode='grayscale',
    label_mode='categorical',
    batch_size=32,
    image_size=(28, 28),
    seed=2023,
)

Found 27455 files belonging to 24 classes.
Found 7172 files belonging to 24 classes.


In [55]:
test_data = dataset_train.take(1)
for image, label in test_data.as_numpy_iterator():
    print(label.shape)
    print(image.shape)
    
    print(image.dtype)

(32, 24)
(32, 28, 28, 1)
float32


# Modelling!

In [78]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28,28,1), dtype=tf.float32),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(24)
])

loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True,
)

optim = tf.keras.optimizers.Adam(
    learning_rate=1e-3
)

model.compile(
    optimizer=optim,
    loss=loss,
    metrics=['accuracy']
)

In [79]:
model.fit(
    x=dataset_train,
    validation_data=dataset_test,
    
    epochs=5
)

Epoch 1/5
858/858 [==============================] - 11s 12ms/step - loss: 7.8728 - accuracy: 0.3724 - val_loss: 1.8372 - val_accuracy: 0.4629
Epoch 2/5
858/858 [==============================] - 10s 12ms/step - loss: 1.1722 - accuracy: 0.6368 - val_loss: 1.8358 - val_accuracy: 0.5109
Epoch 3/5
858/858 [==============================] - 10s 12ms/step - loss: 0.8373 - accuracy: 0.7331 - val_loss: 2.0367 - val_accuracy: 0.5237
Epoch 4/5
858/858 [==============================] - 10s 12ms/step - loss: 0.6032 - accuracy: 0.8060 - val_loss: 1.4185 - val_accuracy: 0.6364
Epoch 5/5
858/858 [==============================] - 10s 12ms/step - loss: 0.5402 - accuracy: 0.8272 - val_loss: 1.7456 - val_accuracy: 0.6323


## Let's see the impact of Data Augmentation

https://www.tensorflow.org/guide/keras/preprocessing_layers#image_data_augmentation_2

In [83]:
dataset_train_augmented = tf.keras.utils.image_dataset_from_directory(
    directory=f"{DATA_PATH}/Train",
    color_mode='grayscale',
    label_mode='categorical',
    batch_size=32,
    image_size=(28, 28),
    seed=2023,
)

Found 27455 files belonging to 24 classes.


In [84]:
data_augmentation_layers = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ]
)

dataset_train_augmented = dataset_train_augmented\
                            .map(lambda x, y: (data_augmentation_layers(x), y))

In [85]:
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28,28,1), dtype=tf.float32),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(24)
])

loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True,
)

optim = tf.keras.optimizers.Adam(
    learning_rate=1e-3
)

model.compile(
    optimizer=optim,
    loss=loss,
    metrics=['accuracy']
)

model.fit(
    x=dataset_train_augmented,
    validation_data=dataset_test,
    epochs=5
)

Epoch 1/5
858/858 [==============================] - 16s 18ms/step - loss: 10.1439 - accuracy: 0.1922 - val_loss: 2.2956 - val_accuracy: 0.3349
Epoch 2/5
858/858 [==============================] - 16s 19ms/step - loss: 2.1260 - accuracy: 0.3287 - val_loss: 2.0710 - val_accuracy: 0.3753
Epoch 3/5
858/858 [==============================] - 16s 19ms/step - loss: 1.8160 - accuracy: 0.4105 - val_loss: 1.8257 - val_accuracy: 0.4449
Epoch 4/5
858/858 [==============================] - 16s 18ms/step - loss: 1.6024 - accuracy: 0.4744 - val_loss: 1.4767 - val_accuracy: 0.5046
Epoch 5/5
858/858 [==============================] - 15s 18ms/step - loss: 1.3508 - accuracy: 0.5478 - val_loss: 1.2922 - val_accuracy: 0.5686


In [86]:
model.fit(
    x=dataset_train_augmented,
    validation_data=dataset_test,
    epochs=5
)

Epoch 1/5
858/858 [==============================] - 15s 18ms/step - loss: 1.2601 - accuracy: 0.5804 - val_loss: 1.1807 - val_accuracy: 0.6135
Epoch 2/5
858/858 [==============================] - 16s 18ms/step - loss: 1.1640 - accuracy: 0.6094 - val_loss: 1.3771 - val_accuracy: 0.5782
Epoch 3/5
858/858 [==============================] - 16s 18ms/step - loss: 1.0458 - accuracy: 0.6515 - val_loss: 1.2392 - val_accuracy: 0.6068
Epoch 4/5
858/858 [==============================] - 17s 20ms/step - loss: 0.9610 - accuracy: 0.6830 - val_loss: 1.5072 - val_accuracy: 0.5594
Epoch 5/5
858/858 [==============================] - 18s 21ms/step - loss: 0.8823 - accuracy: 0.7067 - val_loss: 1.6048 - val_accuracy: 0.6022


Hmm menggunakan data augmentation memang membuat model lebih stabil dalam belajar (bisa dilihat bahwa accuracy dan val_accuracy mengikuti selama 9 epoch)